In [1]:
!nvidia-smi

Wed Dec  6 12:58:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/")
!ls

MyDrive


In [4]:
os.chdir("/content/drive/My Drive/dataset")
!ls

adaniport.csv	 data.pth				my_model.h5	       t1.csv
article		 decision_tree_graphivz.png.png		rapidsai-csp-utils     test.csv
credit_card.csv  diabetes.csv				Real.csv	       titanic.csv
crop.csv	 Grocery.csv				sample_sales_data.csv  train.csv
crops.pkl	 Loan.pickle				solar.csv	       twittersentiment.csv
cufile.log	 Mumbai1.csv				stock.pkl	       weather1.csv
cyber.csv	 mumbai_house_prices_prediction.pickle	submission.csv	       winequality.csv


In [5]:
import warnings
warnings.filterwarnings("ignore")

## Pandas , sklearn

In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [79]:
%%time
df = pd.read_csv('credit_card.csv')

CPU times: user 25.3 ms, sys: 1.77 ms, total: 27.1 ms
Wall time: 31.3 ms


In [80]:
df.shape

(32581, 8)

In [81]:
%%time
df.isnull().sum()

CPU times: user 16.2 ms, sys: 0 ns, total: 16.2 ms
Wall time: 17 ms


person_age                 0
person_income              0
person_home_ownership      0
person_emp_length        895
loan_intent                0
loan_grade                 0
loan_amnt                  0
loan_status                0
dtype: int64

In [82]:
%%time
df = df.dropna()

CPU times: user 17.5 ms, sys: 0 ns, total: 17.5 ms
Wall time: 17.3 ms


In [148]:
df1 = df.drop(['loan_grade','person_emp_length'],axis='columns')
df1.head()

,person_age,person_income,person_home_ownership,loan_intent,loan_amnt,loan_status
0,22,59000,RENT,PERSONAL,35000,1
1,21,9600,OWN,EDUCATION,1000,0
2,25,9600,MORTGAGE,MEDICAL,5500,1
3,23,65500,RENT,MEDICAL,35000,1
4,24,54400,RENT,MEDICAL,35000,1


In [149]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df1['loan_intent']= label_encoder.fit_transform(df['loan_intent'])
df1['loan_intent'].unique()

array([4, 1, 3, 5, 2, 0])

In [150]:
df1['person_home_ownership']= label_encoder.fit_transform(df['person_home_ownership'])
df1['person_home_ownership'].unique()

array([3, 2, 0, 1])

In [151]:
%%time
X = df1.drop(columns=['loan_status'],axis=1)
X

CPU times: user 3.06 ms, sys: 0 ns, total: 3.06 ms
Wall time: 2.99 ms


,person_age,person_income,person_home_ownership,loan_intent,loan_amnt
0,22,59000,3,4,35000
1,21,9600,2,1,1000
2,25,9600,0,3,5500
3,23,65500,3,3,35000
4,24,54400,3,3,35000
...,...,...,...,...,...
32576,57,53000,0,4,5800
32577,54,120000,0,4,17625
32578,65,76000,3,2,35000
32579,56,150000,0,4,15000


In [152]:
Y = df1['loan_status']
Y

0        1
1        0
2        1
3        1
4        1
        ..
32576    0
32577    0
32578    1
32579    0
32580    0
Name: loan_status, Length: 31686, dtype: int64

In [153]:
%%time
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=2)

CPU times: user 4.84 ms, sys: 3.23 ms, total: 8.07 ms
Wall time: 8.5 ms


In [154]:
type(Y_train)

pandas.core.series.Series

In [155]:
from sklearn.ensemble import RandomForestClassifier

In [156]:
%%time
classifier= RandomForestClassifier(n_estimators= 10)
classifier.fit(X_train, Y_train)

CPU times: user 240 ms, sys: 1.11 ms, total: 242 ms
Wall time: 237 ms


RandomForestClassifier(n_estimators=10)

In [157]:
X_train_prediction = classifier.predict(X_train)
training_data_accuray = accuracy_score(X_train_prediction,Y_train)

In [158]:
print('Sk Accuracy : ', training_data_accuray )

Sk Accuracy :  0.9835884487928042


## Rapids

In [6]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 883.9 kB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 

In [7]:
import cudf
from cuml import train_test_split
from cuml.ensemble import RandomForestClassifier  as cuRF

In [8]:
import numpy as np

In [107]:
%%time
cu_df = cudf.read_csv('credit_card.csv')

CPU times: user 7.65 ms, sys: 876 µs, total: 8.53 ms
Wall time: 17.5 ms


In [108]:
%%time
cu_df.isnull().sum()

CPU times: user 6.86 ms, sys: 0 ns, total: 6.86 ms
Wall time: 8.17 ms


person_age                 0
person_income              0
person_home_ownership      0
person_emp_length        895
loan_intent                0
loan_grade                 0
loan_amnt                  0
loan_status                0
dtype: int64

In [109]:
%%time
cu_df = cu_df.dropna()

CPU times: user 3.37 ms, sys: 2 µs, total: 3.37 ms
Wall time: 6.01 ms


In [110]:
cu_df.isnull().sum()

person_age               0
person_income            0
person_home_ownership    0
person_emp_length        0
loan_intent              0
loan_grade               0
loan_amnt                0
loan_status              0
dtype: int64

In [111]:
cu_df = cu_df.drop(columns=['person_emp_length','loan_grade'],axis=1)

In [112]:
print(cu_df.dtypes)

person_age                int64
person_income             int64
person_home_ownership    object
loan_intent              object
loan_amnt                 int64
loan_status               int64
dtype: object


In [113]:
type(cu_df)

cudf.core.dataframe.DataFrame

In [114]:
from cuml import preprocessing as cuMLpreprocessing
label_encoder = cuMLpreprocessing.LabelEncoder()

cu_df['loan_intent']= label_encoder.fit_transform(cu_df['loan_intent'])
cu_df['loan_intent'].unique()

0    4
1    1
2    3
3    5
4    2
5    0
Name: loan_intent, dtype: uint8

In [115]:
cu_df

,person_age,person_income,person_home_ownership,loan_intent,loan_amnt,loan_status
0,22,59000,RENT,4,35000,1
1,21,9600,OWN,1,1000,0
2,25,9600,MORTGAGE,3,5500,1
3,23,65500,RENT,3,35000,1
4,24,54400,RENT,3,35000,1
...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,4,5800,0
32577,54,120000,MORTGAGE,4,17625,0
32578,65,76000,RENT,2,35000,1
32579,56,150000,MORTGAGE,4,15000,0


In [116]:
cu_df['person_home_ownership']= label_encoder.fit_transform(cu_df['person_home_ownership'])
cu_df['person_home_ownership'].unique()

0    3
1    2
2    0
3    1
Name: person_home_ownership, dtype: uint8

In [117]:
print(cu_df.dtypes)

person_age               int64
person_income            int64
person_home_ownership    uint8
loan_intent              uint8
loan_amnt                int64
loan_status              int64
dtype: object


In [136]:
cu_df = cu_df.astype('float64')

In [137]:
cu_df['loan_status'] = cu_df['loan_status'].astype('int64')

In [138]:
print(cu_df.dtypes)

person_age               float64
person_income            float64
person_home_ownership    float64
loan_intent              float64
loan_amnt                float64
loan_status                int64
dtype: object


In [139]:
%%time
X = cu_df.drop(columns=['loan_status'],axis=1)
X

CPU times: user 1.67 ms, sys: 0 ns, total: 1.67 ms
Wall time: 1.67 ms


,person_age,person_income,person_home_ownership,loan_intent,loan_amnt
0,22.0,59000.0,3.0,4.0,35000.0
1,21.0,9600.0,2.0,1.0,1000.0
2,25.0,9600.0,0.0,3.0,5500.0
3,23.0,65500.0,3.0,3.0,35000.0
4,24.0,54400.0,3.0,3.0,35000.0
...,...,...,...,...,...
32576,57.0,53000.0,0.0,4.0,5800.0
32577,54.0,120000.0,0.0,4.0,17625.0
32578,65.0,76000.0,3.0,2.0,35000.0
32579,56.0,150000.0,0.0,4.0,15000.0


In [140]:
y = cu_df['loan_status']
y

0        1
1        0
2        1
3        1
4        1
        ..
32576    0
32577    0
32578    1
32579    0
32580    0
Name: loan_status, Length: 31686, dtype: int64

In [141]:
%%time
X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=2)

CPU times: user 10.4 ms, sys: 0 ns, total: 10.4 ms
Wall time: 10.1 ms


In [142]:
type(y_cudf)

cudf.core.series.Series

In [143]:
%%time
classifier1 = cuRF(n_estimators= 10)
classifier1.fit(X_cudf, y_cudf)

CPU times: user 59.6 ms, sys: 31.5 ms, total: 91.1 ms
Wall time: 56.2 ms


RandomForestClassifier()

In [144]:
import cuml

In [145]:
predictions = classifier1.predict ( X_cudf )
cu_score = cuml.metrics.accuracy_score( y_cudf, predictions )

print( " cuml accuracy: ", cu_score )

 cuml accuracy:  0.9008600115776062
